In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
lr = 1e-3
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

[2017-09-02 16:21:47,331] Making new env: CartPole-v0


In [3]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break
                
# some_random_games_first()

In [4]:
# To get training data
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                    
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output])
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    return training_data

In [5]:
initial_population()

Average accepted score:  61.70942408376963
Median accepted score:  58.0
Counter({50.0: 36, 51.0: 29, 56.0: 25, 53.0: 24, 57.0: 22, 58.0: 21, 52.0: 19, 64.0: 19, 55.0: 14, 62.0: 14, 54.0: 13, 69.0: 10, 61.0: 10, 59.0: 10, 68.0: 10, 63.0: 9, 67.0: 9, 70.0: 8, 65.0: 8, 78.0: 7, 60.0: 7, 71.0: 7, 75.0: 6, 73.0: 5, 66.0: 5, 82.0: 5, 86.0: 3, 83.0: 3, 101.0: 2, 79.0: 2, 76.0: 2, 87.0: 2, 92.0: 1, 80.0: 1, 124.0: 1, 88.0: 1, 81.0: 1, 108.0: 1, 125.0: 1, 126.0: 1, 72.0: 1, 112.0: 1, 89.0: 1, 106.0: 1, 84.0: 1, 94.0: 1, 104.0: 1, 90.0: 1})


[[array([ 0.04646577,  0.19132895, -0.03309827, -0.31345396]), [1, 0]],
 [array([ 0.05029235, -0.00330624, -0.03936735, -0.03139018]), [0, 1]],
 [array([ 0.05022623,  0.19235749, -0.03999515, -0.3362294 ]), [0, 1]],
 [array([ 0.05407338,  0.38802512, -0.04671974, -0.64125157]), [1, 0]],
 [array([ 0.06183388,  0.19358452, -0.05954477, -0.36363953]), [1, 0]],
 [array([ 0.06570557, -0.00064283, -0.06681756, -0.09031065]), [0, 1]],
 [array([ 0.06569271,  0.19537006, -0.06862378, -0.40330365]), [1, 0]],
 [array([ 0.06960011,  0.00128508, -0.07668985, -0.13302096]), [1, 0]],
 [array([ 0.06962582, -0.19265936, -0.07935027,  0.13451591]), [0, 1]],
 [array([ 0.06577263,  0.00350424, -0.07665995, -0.18210812]), [0, 1]],
 [array([ 0.06584271,  0.19963465, -0.08030211, -0.49795566]), [1, 0]],
 [array([ 0.06983541,  0.00573129, -0.09026123, -0.23162136]), [1, 0]],
 [array([ 0.06995003, -0.18799275, -0.09489365,  0.031281  ]), [1, 0]],
 [array([ 0.06619018, -0.38163482, -0.09426803,  0.29258044]), [

In [6]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network)
    return model

In [7]:
def train_model(training_data, model=False):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    y = [i[1] for i in training_data]
    
    if not model:
        model = neural_network_model(input_size=len(X[0]))
        
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_cart_pole_v0')
    return model

In [8]:
training_data = initial_population()
model = train_model(training_data)

Training Step: 998  | total loss: 0.66787 | time: 6.878s
| Adam | epoch: 003 | loss: 0.66787 - acc: 0.6095 -- iter: 21248/21286
Training Step: 999  | total loss: 0.67345 | time: 6.900s
| Adam | epoch: 003 | loss: 0.67345 - acc: 0.6048 -- iter: 21286/21286
--


In [9]:
scores = []
choices = []

for each_game in range(100):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
#         env.render()
        if len(prev_obs) == 0:
            action = random.randrange(0, 2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score += reward
        if done:
            break
        
    scores.append(score)

print('Average Score', sum(scores)/len(scores))
print('Choice 1: {}, Choice 0: {}'.format(choices.count(1)/len(choices), choices.count(0)/len(choices)))

Average Score 166.6
Choice 1: 0.5324129651860744, Choice 0: 0.46758703481392555
